In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
os.environ["TF_CUDNN_DETERMINISTIC"] = "true"
os.environ["TF_CUDNN_WORKSPACE_LIMIT_IN_MB"] = "1024"  # Set the workspace limit to 1GB
os.environ["TF_CUDNN_USE_AUTOTUNE"] = "0"
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [ ]:
import warnings
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import mixed_precision
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

print(tf.__version__)
# Enable GPU acceleration
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except:
        pass

In [ ]:
class CustomDataGenerator(ImageDataGenerator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

batch_size = 64
epochs = 20

# Load the data
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

print("Training set size:", train.shape)
print("Test set size:", test.shape)

In [ ]:
# Extract the label column for analysis
Y_train = train["label"]
X_train = train.drop(labels=["label"], axis=1)

# Normalize the grayscale values
X_train = X_train / 255.0
test = test / 255.0

In [ ]:
# Reshape the input data to have a depth of 1 (grayscale images)
X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

# Convert the labels to categorical format
Y_train = to_categorical(Y_train, num_classes=10)

# Split the training data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=2)

# Define the input shape
input_shape = (28, 28, 1)

In [ ]:
# Define the learning rate schedule function
def lr_schedule(epoch):
    lr = 0.001
    if epoch > 20:
        lr *= 0.5e-3
    elif epoch > 10:
        lr *= 1e-3
    print(f"Learning rate: {lr}")
    return lr

In [ ]:
# Define the CNN model architecture
model = Sequential([
    Input(shape=input_shape),
    Conv2D(32, (3, 3), padding="same"),
    BatchNormalization(),
    Activation("relu"),
    Conv2D(32, (3, 3), padding="same"),
    BatchNormalization(),
    Activation("relu"),
    MaxPool2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), padding="same"),
    BatchNormalization(),
    Activation("relu"),
    Conv2D(64, (3, 3), padding="same"),
    BatchNormalization(),
    Activation("relu"),
    MaxPool2D((2, 2)),
    Dropout(0.25),
    Conv2D(128, (3, 3), padding="same"),
    BatchNormalization(),
    Activation("relu"),
    Conv2D(128, (3, 3), padding="same"),
    BatchNormalization(),
    Activation("relu"),
    MaxPool2D((2, 2)),
    Dropout(0.25),
    Conv2D(256, (3, 3), padding="same"),
    BatchNormalization(),
    Activation("relu"),
    Conv2D(256, (3, 3), padding="same"),
    BatchNormalization(),
    Activation("relu"),
    MaxPool2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(1024, kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Activation("relu"),
    Dropout(0.5),
    Dense(512, kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Activation("relu"),
    Dropout(0.5),
    Dense(10, activation="softmax")
])

In [ ]:
# Define the optimizer
optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, clipnorm=1.0)

# Compile the model
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

# Define an image data generator for data augmentation
datagen = CustomDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
)

In [ ]:
# Define callbacks
lr_scheduler = LearningRateScheduler(lr_schedule)
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

In [ ]:
# Train the model with increased GPU usage and callbacks
with tf.device("/device:GPU:0"):
    history = model.fit(
        datagen.flow(X_train, Y_train, batch_size=batch_size),
        epochs=epochs,
        validation_data=(X_val, Y_val),
        verbose=2,
        callbacks=[lr_scheduler, early_stopping]
    )

In [ ]:
results = model.predict(test)
results = np.argmax(results, axis=1)
results = pd.Series(results, name="Label")
submission = pd.concat([pd.Series(range(1, 28001), name="ImageId"), results], axis=1)
submission.to_csv("submission.csv", index=False)

In [ ]:
# Document end